In [1]:
%pip install --upgrade pip
%pip install --upgrade transformers datasets[audio] accelerate

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------------- ---------------- 1.0/1.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.1
    Uninstalling pip-25.1:
      Successfully uninstalled pip-25.1
Note: you may need to restart the kernel to use updated packages.
INFO: pip is looking at multiple versions of datasets[audio] to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.3 MB 3.9 MB/s eta 0:00:03
   ------- -------------------------------- 2.1/11.3 MB 4.2 MB/s eta 0:00:03
   ----------- ---------------------------- 3.1/11.3 MB 4.2 MB/s eta 0:00:02
   ------------- -------------------------- 3.7/11.3 

In [2]:
import os
os.environ["PATH"] += os.pathsep + r"C:\git_repo\GJ_class\ffmpeg-2025-08-18-git-0226b6fb2c-full_build\bin"

In [3]:
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   - -------------------------------------- 0.3/6.2 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.2 MB 1.2 MB/s eta 0:00:05
   -------- ------------------------------- 1.3/6.2 MB 2.2 MB/s eta 0:00:03
   ------------- -------------------------- 2.1/6.2 MB 2.6 MB/s eta 0:00:02
   ---------------- ----------------------- 2.6/6.2 MB 2.6 MB/s eta 0:00:02
   -------------------- ------------------- 3.1/6.2 MB 2.7 MB/s eta 0:00:02
   ------------------------- -------------- 3.9/6.2 MB 2.8 MB/s eta 0:00:01
   ---------------------------- ----------- 4.5/6.2 MB 2.8 MB/s eta 0:00:01
   ------------------------------- -------- 5.0/6.2 MB 2.7 MB/s eta 0:00:01
   --------------------------------- ------ 5.2/6.2 MB 2.7 MB/s eta 0:00:01
   ----------------------------------- ---- 5.5/6.2 MB 2.4 MB/s eta 0:00:01
   ----------------------------------------

In [6]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    # 이 아래로 추가됨
    return_timestamps = True,
    chunk_length_s=10,
    stride_length_s=2
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

sample = "./audio/lsy_audio_2023_58s.mp3"

result = pipe(sample)
# print(result["text"])


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instea

In [7]:
# chunks를 CSV 파일로 저장

start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("./output/lsy_audio_2023_58.csv", index=False, sep="|")
display(df)

,start,end,text
